# RXiv-Maker: YAML Editor


This notebook lets you create and edit your RXiv-Maker `00_CONFIG.yml` file in Google Colab.

---

**Features:**
- Edit article metadata, affiliations, and authors
- Add co-first/corresponding authors and social media
- Assign and manage affiliations for each author
- Live YAML preview and one-click save

---

**You’ll need:**
- Google Colab
- No coding required

---

_Edit the fields below and click **Save YAML** when done._



In [ ]:
# @markdown ### Run to install the dependencies

!pip install ipywidgets pyyaml -q


In [ ]:
# @markdown ### Run to start the RXiv-Maker YAML editor

import os, yaml, copy, re
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime


# --- Paths ---
folder = "/content"
yaml_filename = "00_CONFIG.yml"
yaml_path = os.path.join(folder, yaml_filename)
drive_folder = '/content'
drive_yaml_path = os.path.join(drive_folder, yaml_filename)

# --- Default YAML structure ---
empty_yaml = {
    'title': [{'long': ''}, {'short': ''}, {'lead_author': ''}],
    'date': '',
    'status': 'draft',
    'use_line_numbers': False,
    'footnote_venue': 'none',
    'keywords': [],
    'authors': [],
    'affiliations': [],
    'bibliography': '03_REFERENCES.bib'
}
if not os.path.exists(yaml_path):
    with open(yaml_path, "w") as f:
        yaml.dump(empty_yaml, f, sort_keys=False)

SOCIAL_MEDIA_CHOICES = [
    ("Twitter", "twitter"),
    ("LinkedIn", "linkedin"),
    ("Facebook", "facebook"),
    ("Instagram", "instagram"),
    ("Bluesky", "bluesky"),
    ("ORCID", "orcid"),
    ("Mastodon", "mastodon"),
    ("Personal Website", "web"),
]


def get_affil_shortnames(affils):
    return [a['shortname'] for a in affils if a.get('shortname')]

def get_next_affil_shortname(affils):
    nums = [int(re.sub("[^0-9]", "", a['shortname'])) for a in affils if re.match(r'Affil\d+$', a.get('shortname', ''))]
    next_num = (max(nums) if nums else 0) + 1
    return f"Affil{next_num}"


import os, yaml, copy, re
import ipywidgets as widgets
from IPython.display import display, clear_output

# ---- Utility helpers and constants ----
SOCIAL_MEDIA_CHOICES = [
    ("Twitter", "twitter"),
    ("LinkedIn", "linkedin"),
    ("Facebook", "facebook"),
    ("Instagram", "instagram"),
    ("Bluesky", "bluesky"),
    ("ORCID", "orcid"),
    ("Mastodon", "mastodon"),
    ("Personal Website", "web"),
]

def get_affil_shortnames(affils):
    return [a['shortname'] for a in affils if a.get('shortname')]

def get_next_affil_shortname(affils):
    nums = [int(re.sub("[^0-9]", "", a['shortname'])) for a in affils if re.match(r'Affil\d+$', a.get('shortname', ''))]
    next_num = (max(nums) if nums else 0) + 1
    return f"Affil{next_num}"

# ---- The Editor class ----
class YamlMetadataEditorV5:
    def __init__(self, yaml_path, drive_yaml_path):
        self.yaml_path = yaml_path
        self.drive_yaml_path = drive_yaml_path
        self.reload_from_file()
        self.build()

    def reload_from_file(self, *_):
        with open(self.yaml_path) as f:
            self.state = yaml.safe_load(f) or copy.deepcopy(empty_yaml)
        self.state.setdefault('authors', [])
        self.state.setdefault('affiliations', [])
        self.build()

    def sync_authors_from_widgets(self):
        # Save all visible author widgets into self.state['authors']
        for i, w in enumerate(getattr(self, 'author_widgets', [])):
            if i < len(self.state['authors']):
                self.state['authors'][i]['name'] = w["name"].value
                self.state['authors'][i]['email'] = w["email"].value
                self.state['authors'][i]['orcid'] = w["orcid"].value
                self.state['authors'][i]['co_first_author'] = w["co_first_author"].value
                self.state['authors'][i]['corresponding_author'] = w["corresponding_author"].value

    def build(self):
        # 1. Article Metadata
        self.title_long = widgets.Text(value=self.state['title'][0]['long'], description='Long title:', layout=widgets.Layout(width='60%'))
        self.title_short = widgets.Text(value=self.state['title'][1]['short'], description='Short title:', layout=widgets.Layout(width='40%'))
        self.title_lead = widgets.Text(value=self.state['title'][2]['lead_author'], description='Lead author:', layout=widgets.Layout(width='30%'))

        if self.state['date']:
            try:
                default_date = datetime.date.fromisoformat(self.state['date'])
            except Exception:
                default_date = datetime.date.today()
        else:
            default_date = None
        self.date_picker = widgets.DatePicker(
            value=default_date,
            description='Date:',
            layout=widgets.Layout(width='30%')
        )
        self.status = widgets.Dropdown(value=self.state['status'], options=['draft', 'submitted', 'published'], description='Status:', layout=widgets.Layout(width='19%'))
        self.linenum = widgets.Checkbox(value=self.state['use_line_numbers'], description='Line numbers?', layout=widgets.Layout(width='22%'))
        self.venue = widgets.Dropdown(value=self.state['footnote_venue'], options=['arxiv', 'biorxiv', 'medrxiv', 'none'], description='Venue:', layout=widgets.Layout(width='30%'))
        kw_list = self.state.get('keywords', [])
        self.keyword_boxes = [widgets.Text(value=kw_list[i] if i < len(kw_list) else "", description=f"Keyword {i+1}", layout=widgets.Layout(width="18%")) for i in range(5)]
        self.keywords = widgets.HBox(self.keyword_boxes)
        article_metadata = widgets.VBox([
            widgets.HTML("<h3>Article Metadata</h3>"),
            widgets.HTML("<div style='color:#555;font-size:13px;margin-bottom:8px;'>"
                 "Enter the core information for your article. "
                 "This data will be saved in your YAML file."
                 "</div>"),
            widgets.HBox([self.title_long, self.title_short]),
            widgets.HBox([self.title_lead, self.date_picker, self.status, self.linenum, self.venue]),
            self.keywords
        ])

        # 2. Affiliations Section
        self.affiliation_short = widgets.Text(description="Short name", layout=widgets.Layout(width='20%'))
        self.affiliation_long = widgets.Text(description="Full name", layout=widgets.Layout(width='40%'))
        self.add_affil_btn = widgets.Button(description='Add Affiliation', icon='plus', layout=widgets.Layout(width='180px'))
        self.add_affil_btn.style.button_color = '#76cdf4'
        self.add_affil_btn.on_click(self.add_affiliation)
        self.affiliations_table = widgets.VBox([])
        self.refresh_affiliations_table()
        affiliations = widgets.VBox([
            widgets.HTML("<h3>Affiliations</h3>"),
            widgets.HTML("<div style='color:#555;font-size:13px;margin-bottom:8px;'>"
                 "Each affiliation should have a unique short name (e.g., 'Affil1') "
                 "and a descriptive full name. Authors will link to affiliations by short name."
                 "</div>"),
            self.affiliations_table,
            widgets.HBox([self.affiliation_short, self.affiliation_long, self.add_affil_btn]),
        ])

        # 3. Authors Section
        self.authors_box = widgets.VBox([])
        self.add_author_btn = widgets.Button(description='Add Author', icon='plus', layout=widgets.Layout(width='160px'))
        self.add_author_btn.style.button_color = '#b5e7a0'
        self.add_author_btn.on_click(self.add_author)
        self.refresh_authors()
        authors_section = widgets.VBox([
            widgets.HTML("<h3>Authors</h3>"),
            widgets.HTML("<div style='color:#555;font-size:13px;margin-bottom:8px;'>"
                 "Enter each author’s information. Assign affiliations, author roles, and social links as needed."
                 "</div>"),
            self.authors_box,
            self.add_author_btn
        ])

        # Controls and output
        self.save_btn = widgets.Button(description='Save YAML (local + Drive)', icon='save', button_style='success', layout=widgets.Layout(width='220px'))
        self.save_btn.on_click(self.save_yaml)
        self.reload_btn = widgets.Button(description='Reload YAML', icon='refresh', button_style='info', layout=widgets.Layout(width='140px'))
        self.reload_btn.on_click(self.reload_from_file)
        self.yaml_out = widgets.Textarea(value='', description='YAML Output:', layout=widgets.Layout(width='99%', height='260px'))

        self.ui = widgets.VBox([
            article_metadata,
            widgets.HTML("<hr style='border-top: 4px solid #222; margin: 5px 0 5px 0;'>"),
            affiliations,
            widgets.HTML("<hr style='border-top: 4px solid #222; margin: 5px 0 5px 0;'>"),
            authors_section,
            widgets.HTML("<hr style='border-top: 4px solid #222; margin: 5px 0 5px 0;'>"),
            widgets.HBox([self.save_btn, self.reload_btn]),
            self.yaml_out
        ])

    def refresh_affiliations_table(self):
        rows = []
        for idx, aff in enumerate(self.state['affiliations']):
            short = widgets.Text(value=aff.get('shortname', ''), description='Short name:', layout=widgets.Layout(width='20%'))
            full = widgets.Text(value=aff.get('full_name', ''), description='Full name:', layout=widgets.Layout(width='40%'))
            del_btn = widgets.Button(description='✖', layout=widgets.Layout(width='32px'), button_style='danger')
            def delete_this(btn, idx=idx):
                self.state['affiliations'].pop(idx)
                self.refresh_affiliations_table()
                self.refresh_authors()
            del_btn.on_click(delete_this)
            def save_aff(change, i=idx, short=short, full=full):
                self.state['affiliations'][i]['shortname'] = short.value
                self.state['affiliations'][i]['full_name'] = full.value
            short.observe(save_aff, names='value')
            full.observe(save_aff, names='value')
            rows.append(widgets.HBox([short, full, del_btn]))
        self.affiliations_table.children = rows

    def add_affiliation(self, *_):
        short = self.affiliation_short.value.strip()
        full = self.affiliation_long.value.strip()
        if short and full and short not in get_affil_shortnames(self.state['affiliations']):
            self.state['affiliations'].append({'shortname': short, 'full_name': full})
            self.affiliation_short.value = ""
            self.affiliation_long.value = ""
            self.refresh_affiliations_table()
            self.refresh_authors()

    def refresh_authors(self):
        #self.sync_authors_from_widgets()
        author_widgets = []
        self.author_widgets = []
        authors = self.state['authors']
        affil_shortnames = get_affil_shortnames(self.state['affiliations'])
        for idx, author in enumerate(authors):
            number = widgets.Label(f"{idx+1}.", layout=widgets.Layout(width="28px", align_self='center'))
            moveup = widgets.Button(description='⬆️', layout=widgets.Layout(width='32px'))
            movedown = widgets.Button(description='⬇️', layout=widgets.Layout(width='32px'))
            moveup.disabled = idx == 0
            movedown.disabled = idx == len(authors)-1
            moveup.on_click(lambda btn, idx=idx: self.move_author(idx, -1))
            movedown.on_click(lambda btn, idx=idx: self.move_author(idx, 1))

            name = widgets.Text(value=author.get('name', ''), description='Name:', layout=widgets.Layout(width='20%'))
            email = widgets.Text(value=author.get('email', ''), description='Email:', layout=widgets.Layout(width='20%'))
            orcid = widgets.Text(value=author.get('orcid', ''), description='ORCID:', layout=widgets.Layout(width='20%'))
            cofirst = widgets.Checkbox(value=author.get('co_first_author', False), description='Co-1st', layout=widgets.Layout(width='10%'))
            corr = widgets.Checkbox(value=author.get('corresponding_author', False), description='Corresponding', layout=widgets.Layout(width='10%'))
            del_btn = widgets.Button(description='✖', layout=widgets.Layout(width='32px'), button_style='danger')
            del_btn.on_click(lambda btn, idx=idx: self.delete_author(idx))

            # Author's affiliations (dropdowns)
            affils_widgets = []
            for aidx, affil in enumerate(author.get('affiliations', [])):
                dropdown = widgets.Dropdown(
                    options=affil_shortnames, value=affil if affil in affil_shortnames else None, layout=widgets.Layout(width='10%'))
                def on_change(change, aidx=aidx, idx=idx):
                    if change['name'] == 'value':
                        self.state['authors'][idx]['affiliations'][aidx] = change['new']
                dropdown.observe(on_change, names='value')
                remove_btn = widgets.Button(description='✖', layout=widgets.Layout(width='32px'), button_style='danger')
                remove_btn.on_click(lambda btn, aidx=aidx, idx=idx: self.remove_author_affil(idx, aidx))
                affils_widgets.append(widgets.HBox([dropdown, remove_btn]))
            add_affil_btn = widgets.Button(description='Add affiliation', icon='plus', layout=widgets.Layout(width='130px'))
            add_affil_btn.style.button_color = '#76cdf4'
            add_affil_btn.on_click(lambda btn, idx=idx: self.add_author_affil(idx))
            affil_box = widgets.VBox(affils_widgets + [add_affil_btn])

            # Social media
            social_rows = []
            for sidx, s in enumerate(author.get('social', [])):
                smedia = widgets.Dropdown(options=SOCIAL_MEDIA_CHOICES, value=s.get('media', ''), layout=widgets.Layout(width='120px'))
                shandle = widgets.Text(value=s.get('handle', ''), placeholder="User/URL", layout=widgets.Layout(width='180px'))
                del_soc_btn = widgets.Button(description='✖', layout=widgets.Layout(width='32px'), button_style='danger')
                del_soc_btn.on_click(lambda btn, idx=idx, sidx=sidx: self.remove_social(idx, sidx))
                smedia.observe(lambda change, idx=idx, sidx=sidx, smedia=smedia: self.set_social_media(idx, sidx, 'media', smedia.value), names='value')
                shandle.observe(lambda change, idx=idx, sidx=sidx, shandle=shandle: self.set_social_media(idx, sidx, 'handle', shandle.value), names='value')
                social_rows.append(widgets.HBox([smedia, shandle, del_soc_btn]))
            add_social_btn = widgets.Button(description='Add Social Media', icon='plus', layout=widgets.Layout(width='140px'))
            add_social_btn.style.button_color = '#c5e6a3'
            add_social_btn.on_click(lambda btn, idx=idx: self.add_social(idx))
            social_box = widgets.VBox(social_rows + [add_social_btn])

            # Layout author box
            row = widgets.HBox([
                number, moveup, movedown, name, email, orcid, cofirst, corr, del_btn
            ])

            author_box = widgets.VBox([
                row,
                widgets.HTML("<b>Affiliations:</b>"),
                affil_box,
                widgets.HTML("<b>Social media:</b>"),
                social_box
            ], layout=widgets.Layout(
                border='2px solid #aacfee',
                border_radius='14px',
                padding='14px',
                margin='8px 0 8px 0',
                box_shadow='0 2px 12px #eee',
                background_color='#f8fbff'
            ))

            author_widgets.append(author_box)
            self.author_widgets.append({
                "name": name, "email": email, "orcid": orcid,
                "co_first_author": cofirst, "corresponding_author": corr
            })
        self.authors_box.children = author_widgets

    def move_author(self, idx, direction):
        authors = self.state['authors']
        if 0 <= idx+direction < len(authors):
            authors[idx], authors[idx+direction] = authors[idx+direction], authors[idx]
        self.refresh_authors()

    def add_author(self, *_):
        self.state['authors'].append({
            'name': '', 'affiliations': [],
            'co_first_author': False, 'corresponding_author': False,
            'email': '', 'orcid': '', 'social': []
        })
        self.refresh_authors()

    def delete_author(self, idx):
        self.state['authors'].pop(idx)
        self.refresh_authors()

    def add_author_affil(self, idx):
        self.state['authors'][idx].setdefault('affiliations', []).append("")
        self.refresh_authors()

    def remove_author_affil(self, idx, aidx):
        self.state['authors'][idx]['affiliations'].pop(aidx)
        self.refresh_authors()

    # Social media logic
    def add_social(self, idx):
        self.state['authors'][idx].setdefault('social', []).append({'media': 'twitter', 'handle': ''})
        self.refresh_authors()
    def remove_social(self, idx, sidx):
        self.state['authors'][idx]['social'].pop(sidx)
        self.refresh_authors()
    def set_social_media(self, idx, sidx, field, value):
        self.state['authors'][idx]['social'][sidx][field] = value

    def save_yaml(self, *_):
        # Save state from widgets
        authors_new = []
        for i, aw in enumerate(getattr(self, 'author_widgets', [])):
            if i < len(self.state['authors']):
                author = dict(self.state['authors'][i])  # preserve non-widget fields like affiliations, social
            else:
                author = {
                    'name': '', 'affiliations': [],
                    'co_first_author': False, 'corresponding_author': False,
                    'email': '', 'orcid': '', 'social': []
                }
            author['name'] = aw['name'].value
            author['email'] = aw['email'].value
            author['orcid'] = aw['orcid'].value
            author['co_first_author'] = aw['co_first_author'].value
            author['corresponding_author'] = aw['corresponding_author'].value

            if author['name'].strip() or author['email'].strip() or author['orcid'].strip():
                authors_new.append(author)
        self.state['authors'] = authors_new

        self.state['title'][0]['long'] = self.title_long.value
        self.state['title'][1]['short'] = self.title_short.value
        self.state['title'][2]['lead_author'] = self.title_lead.value
        selected_date = self.date_picker.value
        if selected_date is None:
            selected_date = datetime.date.today()
        self.state['date'] = selected_date.isoformat()
        self.state['status'] = self.status.value
        self.state['use_line_numbers'] = self.linenum.value
        self.state['footnote_venue'] = self.venue.value
        self.state['keywords'] = [box.value.strip() for box in self.keyword_boxes if box.value.strip()]
        self.state['affiliations'] = [a for a in self.state['affiliations'] if a.get('shortname')]
        self.state['bibliography'] = "03_REFERENCES.bib"
        with open(self.yaml_path, "w") as f:
            yaml.dump(self.state, f, sort_keys=False, allow_unicode=True)
        with open(self.drive_yaml_path, "w") as f:
            yaml.dump(self.state, f, sort_keys=False, allow_unicode=True)
        self.yaml_out.value = yaml.dump(self.state, sort_keys=False, allow_unicode=True)
        self.refresh_affiliations_table()
        self.refresh_authors()


    def display(self):
        display(self.ui)

# --- Start the Editor ---
editor = YamlMetadataEditorV5(yaml_path, drive_yaml_path)
editor.display()